In [1]:
import librosa
import numpy as np
import pandas as pd
import parselmouth
from parselmouth.praat import call
from scipy.signal import find_peaks, lfilter, hamming
from scipy.io import wavfile
from scipy.fftpack import fft
import features

In [2]:
# Load an audio file
audio_path = '/Users/imdohyeon/Documents/PythonWorkspace/Lieon-ai/Dataset/data2.wav'  # Replace with your audio file path
y, sr = librosa.load(audio_path, sr=44100)  # 다운샘플링 하지 않음

In [3]:
pitch = features.extract_pitch(y, sr)
pitch

,Pitch
0,171.505035
1,168.562057
2,169.710266
3,773.687805
4,770.447021
...,...
32471,328.506378
32472,328.033783
32473,329.060669
32474,339.433411


In [4]:
f0_pyworld = features.extract_f0_pyworld(y, sr)
f0_pyworld

,0
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
32471,332.452480
32472,326.392799
32473,341.169287
32474,342.547630


In [9]:
# formants = features.extract_formants(audio_path)
formants = features.extract_formants_for_frames(audio_path)
formants

array([[   0.        ,    0.        ,  954.23351941, 2228.71600694,
        3062.92991887],
       [   0.        ,    0.        ,  848.07598219, 2243.19541676,
        3074.67025308],
       [   0.        ,  778.64840647, 1731.56868516, 2689.58550485,
        3110.50944083],
       ...,
       [ 346.45605651, 1100.19197178, 1978.2803354 , 2606.16888886,
        3474.86527336],
       [ 349.46123539, 1074.3199299 , 2005.71411826, 2607.74524111,
        3602.22863511],
       [ 350.88821363, 1080.64058376, 2094.28130233, 3023.03426004,
        3859.47610015]])

In [ ]:
# 함수 사용 예시
formants = features.extract_formants_praat(audio_path)
formants

In [ ]:
spectral_flux = features.extract_spectral_flux(y, sr)
spectral_flux

In [ ]:
spectral_entropy = features.extract_spectral_entropy(y, sr)
spectral_entropy

In [3]:
speech_rate, pause_durations = features.extract_prosody_features(y, sr)
speech_rate

,Speech_Rate
0,40.0
1,40.0
2,40.0
3,40.0
4,40.0
...,...
37696,40.0
37697,40.0
37698,40.0
37699,40.0


In [4]:
pause_durations

,Average_Pause
0,0
1,0
2,0
3,0
4,0
...,...
32468,0
32469,0
32470,0
32471,0


In [ ]:
mfcc = features.extract_mfcc(y, sr)
mfcc